<img src="https://storage.googleapis.com/gweb-uniblog-publish-prod/images/gemma-header.width-1200.format-webp.webp" width="100%">

## Instruct Fine-tuning [Gemma](https://blog.google/technology/developers/gemma-open-models/) using qLora and Supervise Finetuning

This is a comprahensive notebook and tutorial on how to fine tune the `gemma-7b-it` Model

All the code will be available on my Github. Do drop by and give a follow and a star.
[adithya-s-k](https://github.com/adithya-s-k)
\
[Github Code](https://github.com/adithya-s-k/LLM-Cookbook/blob/main/LLMs/Gemma/finetune-gemma.ipynb)

I also post content about LLMs and what I have been working on Twitter.
[AdithyaSK (@adithya_s_k) / X](https://twitter.com/adithya_s_k)

## Prerequisites

Before delving into the fine-tuning process, ensure that you have the following prerequisites in place:

1. **GPU**: [gemma-2b](https://huggingface.co/google/gemma-2b) - can be finetuned on T4(free google colab) while [gemma-7b](https://huggingface.co/google/gemma-7b) requires an A100 GPU.
2. **Python Packages**: Ensure that you have the necessary Python packages installed. You can use the following commands to install them:

Let's begin by checking if your GPU is correctly detected:

In [ ]:
!nvidia-smi

Sat Apr 20 07:38:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Step 2 - Model loading
We'll load the model using QLoRA quantization to reduce the usage of memory


In [ ]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 8.3 MB/s eta 0:00:00
     ━━━━

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

Now we specify the model ID and then we load it with our previously defined quantization configuration.Now we specify the model ID and then we load it with our previously defined quantization configuration.

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model_id = "google/gemma-7b-it"
# model_id = "google/gemma-7b"
# model_id = "google/gemma-2b-it"
# model_id = "google/gemma-2b"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  <start_of_turn>user
  Generate an advertisement campaign campaign given the following information.
  {query}
  <end_of_turn>\n<start_of_turn>model


  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  # decoded = tokenizer.batch_decode(generated_ids)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return (decoded)

In [ ]:
result = get_completion(query="code the fibonacci series in python using reccursion", model=model, tokenizer=tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



  user
  Generate an advertisement campaign campaign given the following information.
  code the fibonacci series in python using reccursion
  
model


    # Fibonacci Series Generator using Recursion in Python

  # Image: A spiral staircase of numbers, representing the fibonacci sequence.

  # Headline: "Unravel the Secrets of the Fibonacci Sequence with Python"

  # Subheading: "Discover the power of recursion and generate endless Fibonacci numbers."

  # Body Text:

  "The Fibonacci sequence, a timeless mathematical phenomenon, has captivated mathematicians and programmers alike. Write Python code that captures the essence of this sequence through recursion, a technique that enables functions to call themselves.

  With recursion, you can generate an infinite number of Fibonacci numbers, each one being the sum of the previous two numbers in the sequence. This method fosters a deeper understanding of mathematics and programming, all while delivering a streamlined solution.

  Here's

## Step 3 - Load dataset for finetuning

### Lets Load the Dataset

For this tutorial, we will fine-tune Mistral 7B Instruct for code generation.

We will be using this [dataset](https://huggingface.co/datasets/TokenBender/code_instructions_122k_alpaca_style) which is curated by [TokenBender (e/xperiments)](https://twitter.com/4evaBehindSOTA) and is an excellent data source for fine-tuning models for code generation. It follows the alpaca style of instructions, which is an excellent starting point for this task. The dataset structure should resemble the following:

```json
{
  "instruction": "Create a function to calculate the sum of a sequence of integers.",
  "input": "[1, 2, 3, 4, 5]",
  "output": "# Python code def sum_sequence(sequence): sum = 0 for num in sequence: sum += num return sum"
}
```

In [ ]:
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving final.csv to final.csv


In [ ]:
# Load the dataset
df = pd.read_csv('final.csv')

In [ ]:
df.head(10)

,company_name,company_message,car_model,car_description,car_price,type_mode_of_campaign,date_of_campaign,description_of_campaign,concept_of_campaign,slogan_of_campaign,target_audience_of_campaign,reason
0,Toyota,**Introducing the Toyota Innova Crysta: The Ep...,Innova Crysta,The Innova ?Crysta? from the brand Innova ? sy...,Rs. 19.99 Lakh - Rs. 26.30 Lakh,**Online Campaigns:**\n\n* **Website and Socia...,NaN,Taking ahead the success story of first 3 phas...,"**Campaign Concept: ""Embark on a Legendary Jou...",**Bold and Boundless: Unleash the Spirit of Ex...,The campaign is specifically targeted at Gover...,Drive the Nation campaign has been highly appr...
1,Toyota,**Headline: Embark on Unforgettable Journeys w...,Innova Crysta,The limited-edition Innova Crysta offers a 360...,Rs 17.17 lakh and Rs 24.99 lakh (ex-showroom),**Offline Marketing Campaigns:**\n\n* **Televi...,19-Oct-21,Toyota Kirloskar Motor (TKM) launched the Inno...,Drive the Nation,* Embark on an Extraordinary Journey\n* Conque...,festive buyers,**Reasons for Success:**\n\n* **Strong brand r...
2,Toyota,Witness the Era of HY with the all-new Innova ...,Innova HyCross,"Introducing the all-new Toyota Innova HyCross,...",Rs. 18.55 Lakh to 29.99 Lakh,Online,NaN,**Campaign Name:** Toyota Innova HyCross: Expe...,Introducing the Era of HY,**Elevate Your Journey with the Toyota Innova ...,**Primary Target Audience:**\n\n* **Urban fami...,**Reasons for Success**\n\n* **Spacious and ve...
3,Kia,**Introducing the All-New Kia Carnival MPV: Fa...,Carnival MPV,The Carnival MPV redefines the category and pu...,"$34,995-$48,095",Multifaceted Marketing Campaign,08-Apr-21,Kia is debuting the all-new 2022 Carnival MPV ...,The Carnival MPV defines a new vehicle segment...,"'What Else Ya Got?""","Active consumers who need a versatile, capable...",**Reasons for Success:**\n\n* **Unique Value P...
4,Kia,"Now, experience the brand new features of the ...",Kia Carens,The Kia Carens gives consumers an experience o...,NaN,digital campaign,17-Feb-22,The campaign surrounds the brand message for t...,The campaign captures the insight that the fea...,It?s from a different world.,youth,**Reasons for Success:**\n\n* **Innovative des...
5,Maruti Suzuki,**Headline:** Experience the Versatility of Ma...,Ertiga,Redesigned,?8.69 Lakh to ?13.03 Lakh,Targeted marketing,30-Mar-16,Used Pairing SDK technology to target users ex...,Second-screening,Move together in Style,Users who consume content on smart devices and...,**Reasons for Success:**\n\n* **Spacious and p...
6,Suzuki,**Headline:** Experience Versatility and Comfo...,Ertiga,The Suzuki Ertiga is a high-spec model with ch...,under the R300 000 mark,"""All-in? marketing",08-Mar-19,Suzuki launched the Ertiga to South Africa?s m...,"""All-in?",#VALUE!,families,"From social media, to email campaigns and a va..."
7,Maruti Suzuki,**Headline:** Experience the Smart & Spacious ...,Ertiga,The Next-Gen Ertiga is positioned as a ?Stylis...,? 8.69 lakh,Brand campaign,15-Dec-18,"Dentsu Impact, the creative agency from Dentsu...",This campaign saw the team truly working toget...,Move together in Style?,"consumers, on offline and digital platforms th...",**Reasons for Success:**\n\n* **Value for mone...
8,Maruti Suzuki,**Headline:** Experience the Versatile Ertiga ...,Ertiga,**Maruti Suzuki Ertiga: A Versatile and Practi...,NaN,television,08-Dec-17,The ads titled 'Real Time Social' promote the ...,The ads follow four friends on a road trip awa...,Move Together in Style,**Primary Target Audience:**\n\n* **Families:*...,**Reasons for Success:**\n\n* **Spacious and p...
9,Hakuhodo,**Company Message: Hakuhodo XL6**\n\n**Our Mis...,XL6,A world of comfort. A world of style. A world ...,NaN,360-degree,26-Aug-19,The 360-degree #FeelsJustRight campaign includ...,A world of comfort and style. A world that fee...,#FeelsJustRight,People who are looking for a comfortable and s...,**Reasons for Success**\n\n* **Innovative and ...


In [ ]:
# Combine the input columns into a single text with special tokens
df['text'] = '[COMPANY]' + df['company_name'] + ' [MESSAGE]' + df['company_message'] + ' [MODEL]' + df['car_model'] + ' [DESCRIPTION]' + df['car_description'] + ' [PRICE]' + df['car_price'].astype(str) + ' [MODE OF CAMPAIGN]' + df['type_mode_of_campaign']

In [ ]:
df['target'] = '[DESCRIPTION]' + df['description_of_campaign'] + ' [CONCEPT]' + df['concept_of_campaign'] + ' [SLOGAN]' + df['slogan_of_campaign'] + ' [TARGET AUDIENCE]' + df['target_audience_of_campaign'] + ' [REASON]' + df['reason']

In [ ]:
df.head()

,company_name,company_message,car_model,car_description,car_price,type_mode_of_campaign,date_of_campaign,description_of_campaign,concept_of_campaign,slogan_of_campaign,target_audience_of_campaign,reason,text,target
0,Toyota,**Introducing the Toyota Innova Crysta: The Ep...,Innova Crysta,The Innova ?Crysta? from the brand Innova ? sy...,Rs. 19.99 Lakh - Rs. 26.30 Lakh,**Online Campaigns:**\n\n* **Website and Socia...,NaN,Taking ahead the success story of first 3 phas...,"**Campaign Concept: ""Embark on a Legendary Jou...",**Bold and Boundless: Unleash the Spirit of Ex...,The campaign is specifically targeted at Gover...,Drive the Nation campaign has been highly appr...,[COMPANY]Toyota [MESSAGE]**Introducing the Toy...,[DESCRIPTION]Taking ahead the success story of...
1,Toyota,**Headline: Embark on Unforgettable Journeys w...,Innova Crysta,The limited-edition Innova Crysta offers a 360...,Rs 17.17 lakh and Rs 24.99 lakh (ex-showroom),**Offline Marketing Campaigns:**\n\n* **Televi...,19-Oct-21,Toyota Kirloskar Motor (TKM) launched the Inno...,Drive the Nation,* Embark on an Extraordinary Journey\n* Conque...,festive buyers,**Reasons for Success:**\n\n* **Strong brand r...,[COMPANY]Toyota [MESSAGE]**Headline: Embark on...,[DESCRIPTION]Toyota Kirloskar Motor (TKM) laun...
2,Toyota,Witness the Era of HY with the all-new Innova ...,Innova HyCross,"Introducing the all-new Toyota Innova HyCross,...",Rs. 18.55 Lakh to 29.99 Lakh,Online,NaN,**Campaign Name:** Toyota Innova HyCross: Expe...,Introducing the Era of HY,**Elevate Your Journey with the Toyota Innova ...,**Primary Target Audience:**\n\n* **Urban fami...,**Reasons for Success**\n\n* **Spacious and ve...,[COMPANY]Toyota [MESSAGE]Witness the Era of HY...,[DESCRIPTION]**Campaign Name:** Toyota Innova ...
3,Kia,**Introducing the All-New Kia Carnival MPV: Fa...,Carnival MPV,The Carnival MPV redefines the category and pu...,"$34,995-$48,095",Multifaceted Marketing Campaign,08-Apr-21,Kia is debuting the all-new 2022 Carnival MPV ...,The Carnival MPV defines a new vehicle segment...,"'What Else Ya Got?""","Active consumers who need a versatile, capable...",**Reasons for Success:**\n\n* **Unique Value P...,[COMPANY]Kia [MESSAGE]**Introducing the All-Ne...,[DESCRIPTION]Kia is debuting the all-new 2022 ...
4,Kia,"Now, experience the brand new features of the ...",Kia Carens,The Kia Carens gives consumers an experience o...,NaN,digital campaign,17-Feb-22,The campaign surrounds the brand message for t...,The campaign captures the insight that the fea...,It?s from a different world.,youth,**Reasons for Success:**\n\n* **Innovative des...,"[COMPANY]Kia [MESSAGE]Now, experience the bran...",[DESCRIPTION]The campaign surrounds the brand ...


Instruction Fintuning - Prepare the dataset under the format of "prompt" so the model can better understand :
1. the function generate_prompt : take the instruction and output and generate a prompt
2. shuffle the dataset
3. tokenizer the dataset

### Formatting the Dataset

Now, let's format the dataset in the required [gemma instruction formate](https://huggingface.co/google/gemma-7b-it).

> Many tutorials and blogs skip over this part, but I feel this is a really important step.

```
<start_of_turn>user What is your favorite condiment? <end_of_turn>
<start_of_turn>model Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavor to whatever I'm cooking up in the kitchen!<end_of_turn>
```

You can use the following code to process your dataset and create a JSONL file in the correct format:

In [ ]:
for datapoint in df:
  print(datapoint)

company_name
company_message
car_model
car_description
car_price
type_mode_of_campaign
date_of_campaign
description_of_campaign
concept_of_campaign
slogan_of_campaign
target_audience_of_campaign
reason
text
target


In [ ]:
def generate_prompt(data_point):
    """Gen. input text based on a prompt, task instruction, (context info.), and answer

    :param data_point: dict: Data point
    :return: dict: tokenzed prompt
    """
    # prefix_text = 'Below is an instruction that describes a task. Write a response that ' \
    #            'appropriately completes the request.\n\n'
    # Samples with additional context into.
    if data_point['text']:
        text = f"""<start_of_turn>user {data_point["text"]} here is the description of the company, the car and required type of campaign to be generated  {data_point["text"]} <end_of_turn>\n<start_of_turn>model{data_point["target"]} <end_of_turn>"""
    # Without
    else:
        text = f"""<start_of_turn>user{data_point["text"]}  here is the description of the company, the car and required type of campaign to be generated <end_of_turn>\n<start_of_turn>model{data_point["target"]} <end_of_turn>"""
    return text

# add the "prompt" column in the dataset
text_column = [generate_prompt(data_point) for data_point in df]
df = df.add_column("prompt", text_column)

TypeError: string indices must be integers

We'll need to tokenize our data so the model can understand.


In [ ]:
dataset = dataset.shuffle(seed=1234)  # Shuffle dataset here
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

Split dataset into 90% for training and 10% for testing

In [ ]:
dataset = dataset.train_test_split(test_size=0.2)
train_data = dataset["train"]
test_data = dataset["test"]

### After Formatting, We should get something like this

```json
{
"text":"<start_of_turn>user Create a function to calculate the sum of a sequence of integers. here are the inputs [1, 2, 3, 4, 5] <end_of_turn>
<start_of_turn>model # Python code def sum_sequence(sequence): sum = 0 for num in sequence: sum += num return sum <end_of_turn>",
"instruction":"Create a function to calculate the sum of a sequence of integers",
"input":"[1, 2, 3, 4, 5]",
"output":"# Python code def sum_sequence(sequence): sum = 0 for num in,
 sequence: sum += num return sum",
"prompt":"<start_of_turn>user Create a function to calculate the sum of a sequence of integers. here are the inputs [1, 2, 3, 4, 5] <end_of_turn>
<start_of_turn>model # Python code def sum_sequence(sequence): sum = 0 for num in sequence: sum += num return sum <end_of_turn>"

}
```

While using SFT (**[Supervised Fine-tuning Trainer](https://huggingface.co/docs/trl/main/en/sft_trainer)**) for fine-tuning, we will be only passing in the “text” column of the dataset for fine-tuning.

In [ ]:
print(test_data)

Dataset({
    features: ['input', 'output', 'text', 'instruction', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 24392
})


## Step 4 - Apply Lora  
Here comes the magic with peft! Let's load a PeftModel and specify that we are going to use low-rank adapters (LoRA) using get_peft_model utility function and  the prepare_model_for_kbit_training method from PEFT.

In [ ]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
print(model)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 3072, padding_idx=0)
    (layers): ModuleList(
      (0-27): 28 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=3072, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=24576, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=24576, bias=False)
          (down_proj): Linear4bit(in_features=24576, out_features=3072, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
   

In [ ]:
import bitsandbytes as bnb
def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [ ]:
modules = find_all_linear_names(model)
print(modules)

['o_proj', 'q_proj', 'up_proj', 'v_proj', 'k_proj', 'down_proj', 'gate_proj']


In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [ ]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 200015872 | total: 8737696768 | Percentage: 2.2891%


## Step 5 - Run the training!

Setting the training arguments:
* for the reason of demo, we just ran it for few steps (100) just to showcase how to use this integration with existing tools on the HF ecosystem.

In [ ]:
# import transformers

# tokenizer.pad_token = tokenizer.eos_token


# trainer = transformers.Trainer(
#     model=model,
#     train_dataset=train_data,
#     eval_dataset=test_data,
#     args=transformers.TrainingArguments(
#         per_device_train_batch_size=1,
#         gradient_accumulation_steps=4,
#         warmup_steps=0.03,
#         max_steps=100,
#         learning_rate=2e-4,
#         fp16=True,
#         logging_steps=1,
#         output_dir="outputs_mistral_b_finance_finetuned_test",
#         optim="paged_adamw_8bit",
#         save_strategy="epoch",
#     ),
#     data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
# )


### Fine-Tuning with qLora and Supervised Fine-Tuning

We're ready to fine-tune our model using qLora. For this tutorial, we'll use the `SFTTrainer` from the `trl` library for supervised fine-tuning. Ensure that you've installed the `trl` library as mentioned in the prerequisites.

In [ ]:
#new code using SFTTrainer
import transformers

from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=0.03,
        max_steps=100,
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

## Lets start training

In [ ]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/home/adithya/miniconda3/envs/gemma-venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,10.299600
2,6.640600
3,7.763200
4,4.142200
5,3.619800
6,4.840600
7,2.886800
8,1.334500
9,1.177300
10,1.343300


TrainOutput(global_step=100, training_loss=1.1844707012176514, metrics={'train_runtime': 188.6612, 'train_samples_per_second': 2.12, 'train_steps_per_second': 0.53, 'total_flos': 3947993787666432.0, 'train_loss': 1.1844707012176514, 'epoch': 0.0})

 Share adapters on the 🤗 Hub

In [ ]:
new_model = "gemma-Code-Instruct-Finetune-test" #Name of the model you will be pushing to huggingface model hub

In [ ]:
trainer.model.save_pretrained(new_model)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
# Push the model and tokenizer to the Hugging Face Model Hub
merged_model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

## Test out Finetuned Model

In [ ]:
result = get_completion(query="code the fibonacci series in python using reccursion", model=merged_model, tokenizer=tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



  user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  code the fibonacci series in python using reccursion
  
model
  

   a Python program to code the Fibonacci series using recursion. Here's a solution:

```python
def fibonacci(n):
  """Calculates the nth Fibonacci number using recursion.

  Args:
    n: The index of the Fibonacci number to calculate.

  Returns:
    The nth Fibonacci number.
  """

  # Base case: The first two Fibonacci numbers are 0 and 1.
  if n <= 1:
    return n

  # Recursive case: Otherwise, calculate the nth Fibonacci number by adding the previous two numbers.
  else:
    return fibonacci(n-1) + fibonacci(n-2)


# Print the Fibonacci numbers.
for i in range(10):
  print(fibonacci(i))
```

**Explanation:**

* The `fibonacci` function takes an integer `n` as input.
* If `n` is less than or equal to 1, it returns `n` itself, as the base case.
* Otherwise, it calculates `n`-th Fibonacci number recurs